# Python高级特性

## 1.图书管理系统：设计并实现一个图书管理系统，该系统能够管理图书的添加、删除、修改、查询以及借阅与归还操作。具体要求如下：
### 1.自定义包与模块：
- 1）定义一个 Book 类，包含图书的基本属性（如ID、标题、作者、出版年份）和方法（如显示图书信息）。
- 2) 定义一个 Library 类，管理图书的集合（使用列表或集合等数据结构），并实现添加、删除、修改、查询图书以及图书的借阅与归还功能。
### 2.面向对象编程：
- 1）在 Book 类中，确保所有属性都被私有化（使用双下划线前缀），并提供公共的getter和setter方法访问这些属性。
- 2）在 Library 类中，使用面向对象的方法来实现各个功能，如通过方法接收参数来添加或删除图书等。
### 3.文件管理：
- 1）实现图书数据的持久化存储。应实时的将当前图书库的状态保存到文件中（如JSON或CSV格式）。
- 2）程序启动时，应从文件中加载图书库的状态。
### 4.异常处理：
在整个系统中加入必要的异常处理机制，确保用户输入错误或系统内部错误时能够给出合理的提示。可能处理的异常类型包括：文件操作异常（如文件不存在、读写错误等）、数据格式错误（如尝试将非法的数据作为图书属性添加）、逻辑错误（如尝试借阅已被借出的图书）。
### 5.用户交互：
设计一个简单的命令行界面（CLI），允许用户通过输入指令来管理图书和借阅活动。至少支持以下操作：添加图书、删除图书、查询图书、借阅图书、归还图书、列出所有图书、退出。
### 6.测试：
编写单元测试，覆盖 Book 类和 Library 类的主要功能。

In [2]:
class Book:
    def __init__(self, book_id, title, author, publish_year):
        self.__book_id = book_id
        self.__title = title
        self.__author = author
        self.__publish_year = publish_year
        self.__is_borrowed = False

    def get_book_id(self):
        return self.__book_id

    def set_book_id(self, book_id):
        self.__book_id = book_id

    def get_title(self):
        return self.__title

    def set_title(self, title):
        self.__title = title

    def get_author(self):
        return self.__author

    def set_author(self, author):
        self.__author = author

    def get_publish_year(self):
        return self.__publish_year

    def set_publish_year(self, publish_year):
        self.__publish_year = publish_year

    def is_borrowed(self):
        return self.__is_borrowed

    def borrow(self):
        self.__is_borrowed = True

    def return_book(self):
        self.__is_borrowed = False

    def display_info(self):
        print(f"ID: {self.__book_id}, Title: {self.__title}, Author: {self.__author}, Year: {self.__publish_year}, Borrowed: {self.__is_borrowed}")


import json

class Library:
    def __init__(self, file_path):
        self.__books = []
        self.__file_path = file_path
        self.__load_books()

    def add_book(self, book):
        if book.get_book_id() in [b.get_book_id() for b in self.__books]:
            raise ValueError("Book with this ID already exists.")
        self.__books.append(book)

    def remove_book(self, book_id):
        self.__books = [book for book in self.__books if book.get_book_id() != book_id]

    def update_book(self, book_id, title=None, author=None, publish_year=None):
        for book in self.__books:
            if book.get_book_id() == book_id:
                if title:
                    book.set_title(title)
                if author:
                    book.set_author(author)
                if publish_year:
                    book.set_publish_year(publish_year)

    def find_book(self, book_id):
        for book in self.__books:
            if book.get_book_id() == book_id:
                return book
        return None

    def borrow_book(self, book_id):
        book = self.find_book(book_id)
        if book and not book.is_borrowed():
            book.borrow()
        elif book is None:
            raise ValueError("Book not found.")
        else:
            raise ValueError("Book is already borrowed.")

    def return_book(self, book_id):
        book = self.find_book(book_id)
        if book and book.is_borrowed():
            book.return_book()
        else:
            raise ValueError("Book is not borrowed or not found.")

    def list_books(self):
        for book in self.__books:
            book.display_info()

    def save_books(self):
        with open(self.__file_path, 'w') as f:
            json.dump([{'id': book.get_book_id(), 'title': book.get_title(), 'author': book.get_author(), 'publish_year': book.get_publish_year(), 'is_borrowed': book.is_borrowed()} for book in self.__books], f, indent=4)

    def __load_books(self):
        try:
            with open(self.__file_path, 'r') as f:
                books_data = json.load(f)
                for book_data in books_data:
                    new_book = Book(book_data['id'], book_data['title'], book_data['author'], book_data['publish_year'])
                    new_book.borrow() if book_data['is_borrowed'] else None
                    self.__books.append(new_book)
        except FileNotFoundError:
            pass


def main():
    library = Library('books.json')
    while True:
        print("\nCommands: add, remove, update, find, borrow, return, list, exit")
        command = input("Enter command: ").strip().lower()
        if command == 'add':
            book_id = input("Enter book ID: ")
            title = input("Enter title: ")
            author = input("Enter author: ")
            publish_year = input("Enter publish year: ")
            new_book = Book(book_id, title, author, publish_year)
            library.add_book(new_book)
        elif command == 'remove':
            book_id = input("Enter book ID to remove: ")
            library.remove_book(book_id)
        elif command == 'update':
            book_id = input("Enter book ID to update: ")
            title = input("Enter new title (press enter to skip): ")
            author = input("Enter new author (press enter to skip): ")
            publish_year = input("Enter new publish year (press enter to skip): ")
            library.update_book(book_id, title or None, author or None, publish_year or None)
        elif command == 'find':
            book_id = input("Enter book ID to find: ")
            book = library.find_book(book_id)
            if book:
                book.display_info()
            else:
                print("Book not found.")
        elif command == 'borrow':
            book_id = input("Enter book ID to borrow: ")
            try:
                library.borrow_book(book_id)
                print("Book borrowed successfully.")
            except ValueError as e:
                print(e)
        elif command == 'return':
            book_id = input("Enter book ID to return: ")
            try:
                library.return_book(book_id)
                print("Book returned successfully.")
            except ValueError as e:
                print(e)
        elif command == 'list':
            library.list_books()
        elif command == 'exit':
            library.save_books()
            break
        else:
            print("Invalid command.")


if __name__ == "__main__":
    main()


Commands: add, remove, update, find, borrow, return, list, exit
ID: asd, Title: asd, Author: asd, Year: 1234, Borrowed: True

Commands: add, remove, update, find, borrow, return, list, exit
ID: asd, Title: asd, Author: asd, Year: 1234, Borrowed: True

Commands: add, remove, update, find, borrow, return, list, exit


## 2.实现斐波那契数列的迭代器（实现__iter__()和__next__()方法）和生成器（使用yield关键字），体会并说明他们之间有什么区别。

In [5]:
class FibonacciIterator:
    def __init__(self):
        self.a, self.b = 0, 1

    def __iter__(self):
        return self

    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a

fib_iterator = FibonacciIterator()
for num in fib_iterator:
    print(num)
    if num > 100:
        break

1
1
2
3
5
8
13
21
34
55
89
144


In [ ]:
def fibonacci_generator():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

fib_generator = fibonacci_generator()
for num in fib_generator:
    print(num)
    if num > 100:
        break

0
1
1
2
3
5
8
13
21
34
55
89
144
